In [1]:
import requests
from urllib.parse import quote

In [2]:
ship = "Drake Cutlass Blue"

In [3]:
data = requests.get(
    f"https://api.star-citizen.wiki/api/v2/vehicles/{quote(ship)}",
    params={
        "locale": "en_EN",
        "include": "components,hardpoints,shops"
    }
).json()

In [4]:
data = data["data"]

In [11]:
from filters import fix_camelcase, minutes_to_time
from json import dumps

output = ""

def oprint(text=""):
    global output
    output += text + "\n"

for item in data.items():
    match item:
        case "name", type_:
            oprint(f"# Ship Stats: {type_}")
        case "cargo_capacity", capacity:
            oprint(f"Cargo capacity: {capacity} SCU")
        case "sizes", {"length": l, "beam": w, "height": h}:
            oprint(f"Dimensions: {l}m length, {w}m width, {h}m height")
        case "emission", emission:
            oprint(f"Emissions: IR: {emission['ir']}, EM Idle: {emission['em_idle']}, EM Max: {emission['em_max']}")
        case "mass", mass:
            oprint(f"Mass: {mass} kg")
        case "vehicle_inventory", vehicle_inventory:
            oprint(f"Vehicle Inventory: {vehicle_inventory}")
        case "personal_inventory", personal_inventory:
            oprint(f"Personal Inventory: {personal_inventory}")
        case "crew", crew:
            crew_info = f"Crew: Skeleton: {crew['min']}"
            if crew['max'] is not None:
                crew_info += f", Max: {crew['max']}"
            if crew['weapon'] is not None:
                crew_info += f", Combat: {crew['weapon']}"
            if crew['operation'] is not None:
                crew_info += f", Operation: {crew['operation']}"
            oprint(crew_info)
        case "health", health:
            oprint(f"Health: {health}")
        case "shield_hp", shield_hp:
            oprint(f"Shield HP: {shield_hp} (From shield generator)")
        case "speed", speed:
            oprint((f"Speed: SCM: {speed['scm']} m/s, Max: {speed['max']} m/s, "
                       f"0 to SCM: {speed['zero_to_scm']} s, 0 to Max: {speed['zero_to_max']} s, "
                       f"SCM to 0: {speed['scm_to_zero']} s, Max to 0: {speed['max_to_zero']} s"))
        case "fuel", fuel:
            oprint((f"Fuel: Capacity: {fuel['capacity']}L, Intake Rate: {fuel['intake_rate']}L/s, "
                       f"Usage: Main: {fuel['usage']['main']}L/s, Maneuvering: {fuel['usage']['maneuvering']}L/s, "
                       f"Retro: {fuel['usage']['retro']}L/s, VTOL: {fuel['usage']['vtol']}L/s"))
        case "quantum", quantum:
            oprint((f"Quantum: Speed: {quantum['quantum_speed']}km/s, Spool Time: {quantum['quantum_spool_time']} s, "
                       f"Fuel Capacity: {quantum['quantum_fuel_capacity']}L, Range: {quantum['quantum_range']} m"))
        case "agility", agility:
            oprint((f"Agility: Pitch: {agility['pitch']}°/s, Yaw: {agility['yaw']}°/s, Roll: {agility['roll']}°/s, "
                       f"Acceleration: Main: {agility['acceleration']['main']} m/s², "
                       f"Retro: {agility['acceleration']['retro']} m/s², "
                       f"VTOL: {agility['acceleration']['vtol']} m/s², "
                       f"Maneuvering: {agility['acceleration']['maneuvering']} m/s²"))
        case "armor", armor:
            oprint(f"Armor: IR: {armor['signal_infrared']}, EM: {armor['signal_electromagnetic']}, Cross Section: {armor['signal_cross_section']}")
        case "foci", [focus]:
            oprint(f"Focus: {focus}")
        case "foci", foci:
            oprint(f"Foci: {', '.join(foci)}")
        case "type", ship_type:
            oprint(f"Role: {ship_type}")
        case "description", description:
            oprint(f"Description: {description}")
        case "size_class", size_class:
            oprint(f"Size Number: {size_class}")
        case "manufacturer", manufacturer:
            oprint(f"Manufacturer: {manufacturer['name']} ({manufacturer['code']})")
        case "insurance", insurance:  
            oprint((f"Insurance: Claim Time: {minutes_to_time(insurance['claim_time'])}m, Expedite Time: {minutes_to_time(insurance['expedite_time'])}m, "
                       f"Expedite Cost: {insurance['expedite_cost']} aUEC"))
        case "updated_at", updated_at:
            oprint(f"Updated At: {updated_at}")
        case "version", version:
            oprint(f"Version: {version}")
        case "production_status", production_status:
            oprint(f"Production Status: {production_status.replace('-', ' ').title()}")
        case "production_note", production_note if production_note != "None":
            oprint(f"Production Note: {production_note}")
        case "size", size:
            oprint(f"Size: {size.title()}")
        case "msrp", msrp:
            oprint(f"MSRP: ${msrp} USD")
        case "loaner", loaner:
            oprint(f"Loaner Ships: {', '.join(loaner) if loaner else 'None'}")
        case "hardpoints", components:
            oprint("\n## Components")
            complete_uuids = []
            uid = lambda x: hash(dumps(x.get("item", {}) | {"children": x.get("children", [])}))
            for component in components:
                if "item" not in component:
                    continue
                count = lambda search_hash: len([x for x in components if uid(x) == search_hash])
                if uid(component) in complete_uuids:
                    continue
                else:
                   #print("adding")
                    complete_uuids.append(uid(component))
                match component["item"]:
                    case {"type": "MissileLauncher", "name": name, "max_missiles": num_missiles}:
                        qty = count(uid(component))
                        s = "s" if qty > 1 else ""
                        oprint(f"{qty}x {name}{s} carrying {num_missiles} {component['children'][0]['item']['name']}{s} each ({qty*num_missiles} total)")
                        complete_uuids.append(uuid)
                    case {"type": "TurretBase", "name": turret_type}:
                        qty = count(uid(component))
                        weapon = component["children"][0]["item"]
                        wp_s = "s" if qty > 1 else ""
                        s = "s" if qty > 1 else ""
                        weapon_qty = len(component["children"]) #! assumes the turret has all the same weapon (probably true)
                        oprint(f"{qty}x {fix_camelcase(turret_type)}{s} with {weapon_qty}x {weapon['name']}{wp_s}")
                    case {"type": "WeaponGun", "name": name, "size": size}:
                        qty = count(uid(component))
                        s = "s" if qty > 1 else ""
                        oprint(f"{qty}x Size {size} {name}{s}")
                    case {"tags": tags} if "gimbalmount" in tags:
                        qty = count(uid(component))
                        s = "s" if qty > 1 else ""
                        wp = component["children"][0]["item"]
                        print(wp)
                        oprint(f"{qty}x Gimballed Size {wp['size']} {wp['name']}{s}")
                    case {"name": name, "type": role, "size": size, "grade": grade, "class": class_, "description": description} if description:
                        qty = count(uid(component))
                        role = fix_camelcase(role)
                        s = "s" if qty > 1 else ""
                        oprint(f"{qty}x Size {size} {name} {role}{s} ({class_} grade {grade})")
                        complete_uuids.append(uuid)
                    case edge:
                        ...
                        #print(edge)
            oprint()
        case "slug", slug:
            # Ignored as it seems unimportant to an end-user
            ...
        case "class_name", class_name:
            # Ignored as it seems unimportant to an end-user
            ...
        case "uuid", uuid:
            # Ignored as it seems unimportant to an end-user
            ...
        case "id", id:
            # Ignored as it seems unimportant to an end-user
            ...
        case "chassis_id", chassis_id:
            # Ignored as it seems unimportant to an end-user
            ...
        case "shops", shops:
            oprint("## Buyable at")
            for shop in shops:
                oprint(f'{shop["name_raw"]} ({shop["items"][0]["price_calculated"]:,} aUEC)')
            oprint()
        case edge:
            print(f"Unhandled item: {edge}")

{'uuid': '7034e330-3d22-4e89-83cb-c01815dbb875', 'name': 'Scorpion GT-215 Gatling', 'description': 'The Scorpion GT-215 is a hydraulically-driven Gatling-type rotary cannon designed to deliver smaller rounds at a very high rate of fire. The Scorpion is designed to shred armor on very fast targets, sacrificing power for absolute saturation of the target area.', 'size': 2, 'mass': 160, 'grade': None, 'class': None, 'manufacturer_description': 'Gallenson Tactical Systems', 'manufacturer': {'name': 'Gallenson Tactical Systems', 'code': 'GAT', 'link': 'https://api.star-citizen.wiki/api/v2/manufacturers/Gallenson+Tactical+Systems'}, 'type': 'WeaponGun', 'sub_type': 'Gun', 'vehicle_weapon': {'class': None, 'type': 'Ballistic Gatling', 'capacity': 295, 'range': 1540, 'damage_per_shot': 35.2, 'modes': [{'mode': 'Rapid', 'type': 'rapid', 'rpm': 1100, 'ammo_per_shot': 1, 'pellets_per_shot': 1, 'damage_per_second': 645.3333333333334}, {'mode': 'RapidTestFire', 'type': None, 'rpm': None, 'ammo_per_

In [12]:
print(output)

# Ship Stats: Cutlass Blue
Dimensions: 37.5m length, 26.5m width, 11.5m height
Emissions: IR: 10144, EM Idle: 12568, EM Max: 51292
Mass: 226700 kg
Cargo capacity: 12 SCU
Vehicle Inventory: 12
Personal Inventory: 1
Crew: Skeleton: 3, Combat: 1
Health: 33172
Shield HP: 9000 (From shield generator)
Speed: SCM: 198 m/s, Max: 1300 m/s, 0 to SCM: 3.235 s, 0 to Max: 21.24 s, SCM to 0: 11.225 s, Max to 0: 73.7 s
Fuel: Capacity: 360000L, Intake Rate: 20L/s, Usage: Main: 188L/s, Maneuvering: 398L/s, Retro: 0L/s, VTOL: 0L/s
Quantum: Speed: 74485900km/s, Spool Time: 7.75 s, Fuel Capacity: 2500L, Range: 382262996941.896 m
Agility: Pitch: 30°/s, Yaw: 30°/s, Roll: 110°/s, Acceleration: Main: 61.204 m/s², Retro: 17.638 m/s², VTOL: 0 m/s², Maneuvering: 140.548 m/s²
Armor: IR: 1, EM: 1, Cross Section: 1
Focus: Interdiction
Role: combat
Description: Sleek, mean, and royal. The Cutlass Blue adds missiles, a more aggressive engine, and Durasteel holding cells in the cargo bay to the standard model. The Cut